In [44]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2
import chess
import torch
from chess.polyglot import zobrist_hash
from chess import Board, Move
import random
from IPython.display import SVG, clear_output, Image
import time
import numpy as np
from numpy import float64, int64
from typing import Dict, Tuple, Sequence, Self, Optional, Any
from dataclasses import dataclass
from pathlib import Path
import sys
sys.path.append(str(Path.cwd() / ".." / "src"))
from mcts import Node, Edge
from leela_cnn import LeelaCNN, board_to_tensor
import matplotlib.pyplot as plt
from matplotlib import animation
import matplotlib.patches as patches

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
def make_random_move(board: Board) -> Board:
    legal_moves = list(board.legal_moves)
    board.push(random.choice(legal_moves))
    return board

def simulate_game():
    board = Board()
    while not board.is_game_over():
        board = make_random_move(board)
    return board.result()

board = Board()
board = make_random_move(board)
board._repr_svg_

<bound method Board._repr_svg_ of Board('rnbqkbnr/pppppppp/8/8/1P6/8/P1PPPPPP/RNBQKBNR b KQkq - 0 1')>

In [46]:
model = LeelaCNN(block_count=10, filter_count=128)
model

LeelaCNN(
  (input): Sequential(
    (0): Conv2d(18, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (blocks): ModuleList(
    (0-9): 10 x Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (4): SEBlock(
        (layers): Sequential(
          (0): AvgPool2d(kernel_size=8, stride=8, padding=0)
          (1): Flatten(start_dim=1, end_dim=-1)
          (2): Linear(in_features=128, out_features=32, bias=True)
          (3): ReLU()
          (4): Linear(in_features=32, out_features=256, bias=True)
        )
        (sigmoid): Sigmoid()
      )
    )
  )
  (activations): ModuleList(
  

In [47]:
board = Board()
tensor = torch.stack([board_to_tensor(board)])
# tensor.shape
model.train()
model(tensor)

torch.Size([1, 256])
torch.Size([1, 128, 8, 8]) torch.Size([1, 128]) torch.Size([1, 128])
torch.Size([1, 256])
torch.Size([1, 128, 8, 8]) torch.Size([1, 128]) torch.Size([1, 128])
torch.Size([1, 256])
torch.Size([1, 128, 8, 8]) torch.Size([1, 128]) torch.Size([1, 128])
torch.Size([1, 256])
torch.Size([1, 128, 8, 8]) torch.Size([1, 128]) torch.Size([1, 128])
torch.Size([1, 256])
torch.Size([1, 128, 8, 8]) torch.Size([1, 128]) torch.Size([1, 128])
torch.Size([1, 256])
torch.Size([1, 128, 8, 8]) torch.Size([1, 128]) torch.Size([1, 128])
torch.Size([1, 256])
torch.Size([1, 128, 8, 8]) torch.Size([1, 128]) torch.Size([1, 128])
torch.Size([1, 256])
torch.Size([1, 128, 8, 8]) torch.Size([1, 128]) torch.Size([1, 128])
torch.Size([1, 256])
torch.Size([1, 128, 8, 8]) torch.Size([1, 128]) torch.Size([1, 128])
torch.Size([1, 256])
torch.Size([1, 128, 8, 8]) torch.Size([1, 128]) torch.Size([1, 128])


(tensor([[0.1377, 0.4260, 1.2280,  ..., 0.6919, 0.2391, 0.9813]],
        grad_fn=<AddmmBackward0>),
 tensor([[0.3215]], grad_fn=<TanhBackward0>))

In [49]:
# max_frames = 60
# board = Board()
# board_states = [board.copy()]
# while not board.is_game_over() and len(board_states) < max_frames:
#     board = make_random_move(board)
#     board_states.append(board.copy())

# gif_path = Path("data") / "tensor_planes.gif"
# gif_path.parent.mkdir(parents=True, exist_ok=True)

# ani.save(gif_path, writer=animation.PillowWriter(fps=3))
# Image(filename=str(gif_path))